In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [23]:
''' 파일 불러오기 '''
df = pd.read_csv("./산림복지/2019산림복지_본문_노이즈 제거.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [24]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [ ]:
df

In [26]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/6127 [00:00<?, ?it/s]

In [6]:
df_1 = df[:1000]
df_2 = df[1000:2000]
df_3 = df[2000:3000]
df_4 = df[3000:4000]
df_5 = df[4000:]

In [48]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./산림복지/2018산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [2]:
df2 = pd.read_csv("./산림복지/2018산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0
1,momo 2018 12 31 19 52 https blog naver com omj...,0
2,An enemy generally says and believes what he w...,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0
4,Carpe diem 현재를 즐겨라,0
...,...,...
135826,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다,5170
135827,조은영 원광대 교수 미술과 는 2017년을 종합하기에는 수락석출 외의 단어들이 지나...,5170
135828,올해의 사자성어는 3위부터 5위까지는 약 16 대의 고른 분포를 보인 것이 특징이다,5170
135829,4위는 16 5위는 15 1 였다,5170


In [47]:
df_1['split_list'].index

RangeIndex(start=0, stop=1000, step=1)

In [ ]:
for i in tqdm(df_1['split_list'].index):
    for j in range(len(df_1['split_list'][i])):
        split_str= {'split_str': df_1['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
for i in tqdm(df_2['split_list'].index):
    for j in range(len(df_2['split_list'][i])):
        split_str= {'split_str': df_2['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [52]:
for i in tqdm(df_3['split_list'].index):
    for j in range(len(df_3['split_list'][i])):
        split_str= {'split_str': df_3['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)

[2999]


In [ ]:
for i in tqdm(df_4['split_list'].index):
    for j in range(len(df_4['split_list'][i])):
        split_str= {'split_str': df_4['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
for i in tqdm(df_5['split_list'].index):
    for j in range(len(df_5['split_list'][i])):
        split_str= {'split_str': df_5['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [21]:
df2.to_csv("./산림복지/2018산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

## 맞춤법 검사

In [1]:
import pandas as pd
df2 = pd.read_csv("./산림복지/2018산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0
1,momo 2018 12 31 19 52 https blog naver com omj...,0
2,An enemy generally says and believes what he w...,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0
4,Carpe diem 현재를 즐겨라,0
...,...,...
135826,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다,5170
135827,조은영 원광대 교수 미술과 는 2017년을 종합하기에는 수락석출 외의 단어들이 지나...,5170
135828,올해의 사자성어는 3위부터 5위까지는 약 16 대의 고른 분포를 보인 것이 특징이다,5170
135829,4위는 16 5위는 15 1 였다,5170


In [2]:
#- 패키지 임포트
import requests
import json
from tqdm.notebook import tqdm
import re
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


'''맞춤법 검사 함수'''
def ko_grammar(sent):
    global correct_list ; correct_list= []
    #for i in tqdm(range(len(a))):
    for i in tqdm(range(len(sent))):
        try:
            print(i)                                                                               
            response = requests.post('http://164.125.7.61/speller/results', data={'text1': sent[i]})  # 맞춤법 검사 요청 (requests)    
            data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] # 응답에서 필요한 내용 추출 (html 파싱)
            data = json.loads(data) # 파이썬 딕셔너리 형식으로 변환
            orgStr = [err['orgStr'] for err in data['errInfo']] #오류가 담긴 errinfo에서 고쳐야 되는 단어 리스트로 추출 
            correct = [err['candWord'] for err in data['errInfo']] #고쳐진 단어도 리스트로 추출
            splited = sent[i] 

            '''여러 개로 고쳐졌을 때 맨 처음 단어만 선택'''
            for j in range(len(correct)):
                if '|'in correct[j]: #여러 개로 고쳐질 경우 문자열이 '|'로 구분되어 있음
                    com = re.compile('\|')
                    many = com.search(correct[j])
                    correct[j]= correct [j][0:(many.span()[0])]
                '''틀린 문장 고치기'''   
                splited = splited.replace(orgStr[j], correct[j])

        except:      
            splited = sent[i]
        correct_list.append(splited)
        display.clear_output(wait=True) # 출력지우기

Output()

In [3]:
# 데이터프레임을 5000개 단위로 나누기
chunk_size = 5000
num_chunks = len(df2) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df2[start_idx:end_idx]
    chunk_df.reset_index(drop=True, inplace=True)
    dfs[f"df2_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df2_1
df2_2
df2_3
df2_4
df2_5
df2_6
df2_7
df2_8
df2_9
df2_10
df2_11
df2_12
df2_13
df2_14
df2_15
df2_16
df2_17
df2_18
df2_19
df2_20
df2_21
df2_22
df2_23
df2_24
df2_25
df2_26
df2_27
df2_28


In [ ]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df_s = pd.DataFrame(columns = ['split_str', 'org_idx', 'correct_str']) # 새로운 데이터프레임 제작

# ''' 문장 데이터 프레임 저장 '''
# df_s.to_csv('./산림복지/2019산림복지_본문_맞춤법검사.csv', index=False, encoding='UTF-8')

In [4]:
df_s = pd.read_csv("./산림복지/2018산림복지_본문_맞춤법검사.csv", encoding='UTF-8')
df_s

,split_str,org_idx,correct_str
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.
1,momo 2018 12 31 19 52 https blog naver com omj...,0,"mom 2018, 12, 31, 19, 52 https blog NAVER com ..."
2,An enemy generally says and believes what he w...,0,An enemy generally says and believes what he w...
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0,True love is the joy of life 진실한 사랑은 인생의 환희다
4,Carpe diem 현재를 즐겨라,0,Crape idem 현재를 즐겨라.
...,...,...,...
39995,국회 농림축산식품해양수산위원회 소속 박완주 의원 더불어민주당 충남 천안을 이 국정감...,1526,"국회 농림축산식품해양수산위원회 소속 박완주 의원 더불어민주당 충남 천안을, 이 국정..."
39996,표1 하지만 이 중 국내재는 484만 톤으로 목재자급률이 16 4 에 불과하다,1526,표1 하지만 이 중 국내 재는 484만 톤으로 목재 자급률이 16 4 에 불과하다
39997,10년 전인 2008년 자급률 11 9 와 비교하면 다소 높아진 수준이지만 아직도 ...,1526,"10년 전인 2008년 자급률 11, 9 와 비교하면 다소 높아진 수준이지만 아직도..."
39998,나무는 계속해서 성장하고 있고 자원을 활용할 수 있는 여건도 제대로 갖춰져 있지 않...,1526,나무는 계속해서 성장하고 있고 자원을 활용할 수 있는 여건도 제대로 갖춰져 있지 않...


In [5]:
'''맞춤법 검사하기''' 
a = int(len(df_s)/5000) + 1
while a < 38:
    ko_grammar(dfs["df2_%s"%a]['split_str'])
    dfs["df2_%s"%a]['correct_str'] = correct_list
    df_s = pd.concat([df_s,dfs["df2_%s"%a]])
    df_s.to_csv('./산림복지/2018산림복지_본문_맞춤법검사.csv', index=False, encoding='UTF-8')
    a += 1

14


In [13]:
df_s.to_csv('./산림복지/2018산림복지_본문_맞춤법검사.csv', encoding='UTF-8')